In [24]:
import pandas as pd
import os
import datetime
import json
from functools import reduce

In [25]:
ROOT_DIR = r"C:\dev\backend\tasks\samples\cmots"
asof_date = datetime.date(2023, 8, 22)
DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y"))

# Files needed to be refreshed from CMOTS ftp : dbdmfmgr.csv, dbdfundm.csv, dbdprof.csv
# FundID,Person Code,Designation,Fund Manager,Experience,Qualification,Fund,DateFrom,DateTo
DIR_PATH

'C:\\dev\\backend\\tasks\\samples\\cmots\\22082023'

In [26]:
def get_df(dir_path, file_name, asof_date, lst_col, lst_drop_col, rename_col_mapping):
    file_name = os.path.join(dir_path, file_name)
    print(file_name)
    if os.path.isfile(file_name):
        print('Reading the file into df...')
        df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        # print(df.head())
        df = df.drop(lst_drop_col, axis=1)
        df.rename(columns=rename_col_mapping, inplace=True)
    else:
        df = pd.DataFrame()

    return df

In [27]:
# get data for MF Fund Manager Date History
# 1] dbdmfmgr.csv
lst_col = ["MF_COCODE","MF_SCHCODE","SCHTYPCODE","FUNDCODE","AREA","WEF","TODATE","STATUS","SLNO","FLAG"]
lst_used = ["MF_SCHCODE","FUNDCODE","AREA","WEF","TODATE","STATUS", "FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]

new_col = ["FundID", "Person Code", "Area_", "DateFrom", "DateTo", "Status_", "Flag_"]
rename_col_mapping = dict(zip(lst_used, new_col))
#print(rename_col_mapping)

df_dbdmfmgr = get_df(DIR_PATH, "dbdmfmgr.csv", asof_date, lst_col, lst_drop, rename_col_mapping)
# df_dbdmfmgr.sort_values("MF_SCHCODE", inplace=True)
df_dbdmfmgr.head()

C:\dev\backend\tasks\samples\cmots\22082023\dbdmfmgr.csv
Reading the file into df...


,FundID,Person Code,Area_,DateFrom,DateTo,Status_,Flag_
0,22370,692,Equity,01/06/2021,18/08/2023,E,A
1,22369,692,Equity,01/06/2021,18/08/2023,E,A
2,22373,692,Equity,01/06/2021,18/08/2023,E,A
3,22371,692,Equity,01/06/2021,18/08/2023,E,A
4,1726,692,Equity,12/05/2022,18/08/2023,E,A


In [30]:
# get data for MF Fund Manager Profile Details
# 2] dbdprof.csv
lst_col = ["FUNDCODE","QUAL","DESIG","EXPERIENCE","REMARKS","FLAG"]
lst_used = ["FUNDCODE","QUAL","DESIG","EXPERIENCE","REMARKS","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]

new_col = ["FundID", "Qualification", "Designation", "Experience", "Remarks_", "Flag_"]
rename_col_mapping = dict(zip(lst_used, new_col))
#print(rename_col_mapping)

df_dbdprof = get_df(DIR_PATH, "dbdprof.csv", asof_date, lst_col, lst_drop, rename_col_mapping)
# df_dbdprof.sort_values("MF_SCHCODE", inplace=True)
df_dbdprof.head()

[]
{'FUNDCODE': 'FundID', 'QUAL': 'Qualification', 'DESIG': 'Designation', 'EXPERIENCE': 'Experience', 'REMARKS': 'Remarks_', 'FLAG': 'Flag_'}
C:\dev\backend\tasks\samples\cmots\22082023\dbdprof.csv
Reading the file into df...


,FundID,Qualification,Designation,Experience,Remarks_,Flag_
0,1074,"MBA (Finance) &, B-Tech (Mechanical Engineering)",FUND MANAGER, Fund Manager & Research Analyst. - Mahindra ...,NaN,A


In [31]:
# get data for MF Fund Manager Names
# 3] dbdfundm.csv
lst_col = ["FUNDCODE","FUND_MGR","FLAG"]
lst_used = ["FUNDCODE","FUND_MGR","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]

new_col = ["FundID", "Fund Manager", "Flag_"]
rename_col_mapping = dict(zip(lst_used, new_col))
#print(rename_col_mapping)

df_dbdfundm = get_df(DIR_PATH, "dbdfundm.csv", asof_date, lst_col, lst_drop, rename_col_mapping)
# df_dbdfundm.sort_values("MF_SCHCODE", inplace=True)
df_dbdfundm.head()

C:\dev\backend\tasks\samples\cmots\22082023\dbdfundm.csv
Reading the file into df...


,FundID,Fund Manager,Flag_
0,1084,Balakumar B,A


In [ ]:
# Next step should be combining the 3 datasets from above, we would require master dumps atleast once every week/month